# OLS vs Decision-Focused (with Gurobi) — **Unbounded DF only**

This notebook uses **gurobiPy** to solve the optimization problems for:

1. The **ground-truth** decisions \(x^*\) (per-row) by minimizing \(x^2 - yx\) with bounds \(0 \le x \le 1\).
2. **Decision-focused learning (DF-Unbounded)**: optimize \(b_0,b_1\) for \(\sum_t [(b_0+b_1 z_t)^2 - y_t(b_0+b_1 z_t)]\); for evaluation we clip \(x=b_0+b_1 z\) to \([0,1]\).

For the **classical regression** baseline, we use **scikit-learn** OLS with intercept to fit \(x^*\) from \(z\).

**Evaluation** uses the unweighted cost: \(\sum_t (x_t^2 - y_t x_t)\). Lower is better.


## 0) Imports and data

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

try:
    import gurobipy as gp
    from gurobipy import GRB
    GUROBI_OK = True
except Exception as e:
    GUROBI_OK = False
    print("gurobipy is not available in this environment. "
          "Install and ensure a valid Gurobi license to run the optimization cells.")
    print("Error:", e)

# Data
z = np.array([1, 4, 5, 10], dtype=float)
y = np.array([0.5, 2.33, 2.43, 1.88], dtype=float)

## 1) Helpers

In [2]:
def clip01(x: np.ndarray) -> np.ndarray:
    return np.clip(x, 0.0, 1.0)

def total_cost(x: np.ndarray, y: np.ndarray) -> float:
    return float(np.sum(x**2 - y * x))

## 2) Ground-truth $x^*$ with Gurobi (bounded QP)

In [3]:
if not GUROBI_OK:
    raise RuntimeError("gurobipy not available. Please install and license Gurobi to run this cell.")

T = len(z)
m = gp.Model("xstar_qp")
m.Params.OutputFlag = 0

x_vars = m.addVars(T, lb=0.0, ub=1.0, name="x")

obj = gp.QuadExpr()
for t in range(T):
    obj.add(x_vars[t] * x_vars[t])
    obj.add(- y[t] * x_vars[t])

m.setObjective(obj, GRB.MINIMIZE)
m.optimize()

x_star = np.array([x_vars[t].X for t in range(T)], dtype=float)
print("x* from Gurobi:", x_star)
print("Total cost (x*):", total_cost(x_star, y))

Set parameter Username
Set parameter LicenseID to value 2625590
Academic license - for non-commercial use only - expires 2026-02-20
x* from Gurobi: [0.25 1.   1.   0.94]
Total cost (x*): -3.7061


## 3) Classical regression (OLS with intercept) — scikit-learn

In [4]:
X = z.reshape(-1, 1)
ols = LinearRegression(fit_intercept=True)
ols.fit(X, x_star)  # fit to Gurobi's x*
b0_OLS = float(ols.intercept_)
b1_OLS = float(ols.coef_[0])

x_hat_OLS = clip01(ols.predict(X))
print(f"OLS coefficients: b0 = {b0_OLS:.6f}, b1 = {b1_OLS:.6f}")
print(f"Total cost (OLS, clipped): {total_cost(x_hat_OLS, y):.6f}")

OLS coefficients: b0 = 0.476071, b1 = 0.064286
Total cost (OLS, clipped): -3.330898


In [5]:
ols.predict(X)

array([0.54035714, 0.73321429, 0.7975    , 1.11892857])

## 4) Decision-Focused (DF-Unbounded) — Gurobi QP for $(b_0,b_1)$

Solve
\[
\min_{b_0,b_1} \sum_t \big[(b_0 + b_1 z_t)^2 - y_t(b_0 + b_1 z_t)\big].
\]
This yields a linear rule \(x=b_0+b_1 z\). We clip \(x\) to \([0,1]\) for evaluation.


In [6]:
if not GUROBI_OK:
    raise RuntimeError("gurobipy not available. Please install and license Gurobi to run this cell.")

m_df = gp.Model("df_unbounded_qp")
m_df.Params.OutputFlag = 0

b0 = m_df.addVar(lb=-gp.GRB.INFINITY, name="b0")
b1 = m_df.addVar(lb=-gp.GRB.INFINITY, name="b1")

obj_df = gp.QuadExpr()
for t in range(len(z)):
    lin = b0 + b1 * z[t]
    obj_df.add(lin * lin)
    obj_df.add(- y[t] * lin)

m_df.setObjective(obj_df, GRB.MINIMIZE)
m_df.optimize()

b0_DF_unb = float(b0.X)
b1_DF_unb = float(b1.X)
x_hat_DF_unb = clip01(b0_DF_unb + b1_DF_unb * z)

print(f"DF-Unbounded coefficients: b0 = {b0_DF_unb:.6f}, b1 = {b1_DF_unb:.6f}")
print(f"Total cost (DF-Unbounded, clipped): {total_cost(x_hat_DF_unb, y):.6f}")

DF-Unbounded coefficients: b0 = 0.590714, b1 = 0.060357
Total cost (DF-Unbounded, clipped): -3.400292


## 5) Comparison and CSV

In [7]:
def row_costs(x): return x**2 - y * x

df_compare = pd.DataFrame({
    "t": np.arange(1, len(z)+1),
    "z": z,
    "y": y,
    "x_star (Gurobi)": x_star,
    "x_hat_OLS": x_hat_OLS,
    "x_hat_DF_unbounded (clipped)": x_hat_DF_unb,
    "cost(x*)": row_costs(x_star),
    "cost(OLS)": row_costs(x_hat_OLS),
    "cost(DF-unbounded)": row_costs(x_hat_DF_unb),
})

totals = {
    "x*": total_cost(x_star, y),
    "OLS": total_cost(x_hat_OLS, y),
    "DF-unbounded": total_cost(x_hat_DF_unb, y),
}
print("Totals (unweighted cost, lower is better):")
for k,v in totals.items():
    print(f"  {k}: {v:.6f}")

df_compare.to_csv("cost_table_gurobi_unbounded.csv", index=False)
print("Saved CSV to cost_table_gurobi_unbounded.csv")

df_compare

Totals (unweighted cost, lower is better):
  x*: -3.706100
  OLS: -3.330898
  DF-unbounded: -3.400292
Saved CSV to cost_table_gurobi_unbounded.csv


,t,z,y,x_star (Gurobi),x_hat_OLS,x_hat_DF_unbounded (clipped),cost(x*),cost(OLS),cost(DF-unbounded)
0,1,1.0,0.50,0.25,0.540357,0.651071,-0.0625,0.021807,0.098358
1,2,4.0,2.33,1.00,0.733214,0.832143,-1.3300,-1.170786,-1.246431
2,3,5.0,2.43,1.00,0.797500,0.892500,-1.4300,-1.301919,-1.372219
3,4,10.0,1.88,0.94,1.000000,1.000000,-0.8836,-0.880000,-0.880000
